In [1]:
import cv2
import matplotlib.pyplot as plt
import imutils
from itertools import combinations
import math
import numpy as np
import datetime
import os

In [2]:
#Load model files and graph
config_file='MobileNetSSD_deploy.prototxt'
frozen_model = 'MobileNetSSD_deploy.caffemodel'

In [3]:
#Create model object
model = cv2.dnn_DetectionModel(frozen_model, config_file)

In [4]:
#Specify the Labels
classLabels = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]


In [5]:
print(classLabels)

['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']


In [6]:
#set input frame properties
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)

<dnn_Model 000001F7AF9205F0>

#Video_Detection

In [12]:
#Load the Vide0
cap = cv2.VideoCapture('testvideo2.mp4')

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)

fourcc = cv2.VideoWriter_fourcc(*'XVID')#Set properties for Video Output file
out = cv2.VideoWriter('output_vid.mp4',fourcc, 20.0, (640,360))

#Calculting the FPS of rendering video
fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0

#Check if the video is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot Open video")
    
font_scale = 2
font = cv2.FONT_HERSHEY_PLAIN

while True:
    ret, frame = cap.read()
    if ret == False:
        out.release()
        cap.release()
        cv2.destroyAllWindows()
        break
    
    
    #Resize the Frame
    frame = cv2.resize(frame,(640,360),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
    #Calculating the number of frame
    total_frames = total_frames + 1

    #Obtain Frame Properties
    ClassIndex, confidence, bbox = model.detect(frame, confThreshold = 0.55)
    
    print(ClassIndex)
    
    
    
    #If no object is detected
    if(len(ClassIndex) == 0):
        cv2.imshow('Object Detection and Distance', frame)
        out.write(frame)
    
     #if only one object is detected, distance cannot be calculated, so just show a rectangle around it
    if(len(ClassIndex)  ==1):
        cv2.rectangle(frame, bbox, (0,255,0), 2)
        out.write(frame)
        
    
    indices=[]
    
    #If some objects are detected, remove the object if it does not falls within one of the required Labels
    if(len(ClassIndex) != 0):
        ClassIndex = ClassIndex.flatten()
        print(ClassIndex, ClassIndex.shape)
        for i in range(ClassIndex.shape[0]):
            if ClassIndex[i]>80:
                indices.append(i)
                continue
            if classLabels[ClassIndex[i]-1] not in ["person", "car", "motorbike", "bicycle", "bus", "truck", "van"]:
                indices.append(i)
        ClassIndex1 = np.delete(ClassIndex, indices)
        print(ClassIndex1)
        mapped = zip(ClassIndex1.flatten(), confidence.flatten(), bbox)
        print(mapped)
        #ClassIndex= ClassIndex1
    
   
       
        #If the distance between 2 objects is less than a threshold, they will be appended to red_zone list otherwise green_zone list    
        red_zone= []
        green_zone=[]
        for ob1, ob2 in combinations(mapped,2): #using combinations function to find all possible pairs
            print(ob1[2], ob2[2])
            #Calculating the center points of both the objects
            cx1, cy1= ((int(ob1[2][0]+(ob1[2][2]/2)), int(ob1[2][1]+(ob1[2][3])/2)))
            cx2,cy2 = ((int(ob2[2][0]+(ob2[2][2]/2)), int(ob2[2][1]+(ob2[2][3])/2)))
            print(cx1,cy1,cx2,cy2)
            dist = math.sqrt(((cx2-cx1)**2)+((cy2-cy1)**2)) #Calculating Distance between two objects
            print(dist)
            if dist<80: #Threshold distance
                if ob1 not in red_zone:
                    ob1=list(ob1)
                    ob1.append(dist)
                    red_zone.append(ob1)
                if ob2 not in red_zone:
                    ob2=list(ob2)
                    ob2.append(dist)
                    red_zone.append(ob2)
            else:
                if ob1 not in green_zone:
                    ob1=list(ob1)
                    ob1.append(dist)
                    green_zone.append(ob1)
                    print('objects = ',ob1, ob2)
                if ob2 not in green_zone:
                    ob2=list(ob2)
                    ob2.append(dist)
                    green_zone.append(ob2)
        
    if(len(ClassIndex) != 0):                  
        #Show Green rectangle around objects from green_zonelist      
        print('Lenghts = ', len(red_zone), len(green_zone))                   
        for ob1 in  green_zone:
            cv2.rectangle(frame, (ob1[2][0], ob1[2][1], ob1[2][2], ob1[2][3]), (0,255,0), 2)
            #cv2.putText(frame, classLabels[ob1[0]], (ob1[2][0]+10, ob1[2][1]+40), font, fontScale= font_scale, color=(0,255,0), thickness=3) 

        #Show Red rectangle around objects from red_zonelist                 
        for ob2 in  red_zone: 
            cv2.rectangle(frame, (ob2[2][0], ob2[2][1], ob2[2][2], ob2[2][3]), (0,0,255), 2)
            #cv2.putText(frame, classLabels[ob2[0]], (ob2[2][0]+10, ob2[2][1]+40), font, fontScale= font_scale, color=(0,0,255), thickness=3) 

    #Calculating Frames Per Second    
    fps_end_time = datetime.datetime.now()
    time_diff = fps_end_time - fps_start_time
    print('Frames = ',total_frames)
    if time_diff.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames / time_diff.seconds)

    fps_text = "FPS: {:.2f}".format(fps)

    cv2.putText(frame, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)  #Display FPS
    out.write(frame) #Write to output video file
    cv2.imshow('Object Detection and Distance', frame)                     
    
    if cv2.waitKey(2) & 0xFF == ord('q'): #close the program on pressing 'q'
        break
        
cap.release()#release input video
out.release()#release output video
cv2.destroyAllWindows()

[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  1
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  2
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  3
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  4
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  5
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  6
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  7
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  8
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  9
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  10
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  11
[[4]]
[4] (1,)
[]
Lenghts =  0 0
Frames =  12
()
Frames =  13
()
Frames =  14
()
Frames =  15
()
Frames =  16
[[4]]
[4] (1,)
[]
Lenghts =  0 0
Frames =  17
()
Frames =  18
()
Frames =  19
()
Frames =  20
()
Frames =  21
()
Frames =  22
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  23
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  24
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  25
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  26
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  27
[[7]]
[7] (1,)
[7]
Le

[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  198
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  199
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  200
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  201
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  202
()
Frames =  203
()
Frames =  204
()
Frames =  205
()
Frames =  206
()
Frames =  207
()
Frames =  208
()
Frames =  209
()
Frames =  210
()
Frames =  211
()
Frames =  212
()
Frames =  213
()
Frames =  214
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  215
[[7]]
[7] (1,)
[7]
Lenghts =  0 0
Frames =  216
[[7]
 [7]]
[7 7] (2,)
[7 7]
[142 247  64  52] [339 206  54  40]
174 273 366 226
197.6689151080665
objects =  [7, 0.7238462, array([142, 247,  64,  52], dtype=int32), 197.6689151080665] (7, 0.605632, array([339, 206,  54,  40], dtype=int32))
Lenghts =  0 2
Frames =  217
[[7]
 [7]]
[7 7] (2,)
[7 7]
[142 247  64  52] [339 206  54  40]
174 273 366 226
197.6689151080665
objects =  [7, 0.72068405, array([142, 247,  64,  52], dtype=int32), 197.6689151080665] (7, 